In [1]:
import pandas as pd

In [4]:
%cd /scratch/s/shreejoy/nxu/CIHR

/gpfs/fs0/scratch/s/shreejoy/nxu/CIHR


In [1]:
!hostname

nia-login05.scinet.local


In [48]:
sig_intron_attr = pd.read_csv("proc/inspect.csv")
sig_intron_attr = sig_intron_attr\
    .assign(
        score = 0, 
        thickStart = sig_intron_attr.start, 
        thickEnd = sig_intron_attr.end,
        itemRgb = "0,0,0",
        blockSizes = sig_intron_attr.end - sig_intron_attr.start,
        blockCount = 1)
blockStarts = sig_intron_attr.groupby("intron_group", as_index=False).apply(lambda x: x.start - x.start.min()).reset_index(level=0)
blockStarts.rename(columns={"start": "blockStarts"}, inplace=True)

sig_intron_attr = pd.merge(sig_intron_attr, blockStarts["blockStarts"], left_index=True, right_index=True)

In [50]:
bed = sig_intron_attr.groupby("intron_group") \
    .agg({
        "chromosome": "first",
        "start": "first",
        "end": "first",
        "intron_group": "first",
        "score": "first",
        "strand": "first",
        "thickStart": "first",
        "thickEnd": "first",
        "itemRgb": "first",
        "blockCount": "count",
        "blockSizes": lambda x: ",".join(x.astype(str)),
        "blockStarts": lambda x: ",".join(x.astype(str))
    })

In [51]:
bed.to_csv("proc/inspect.bed", sep="\t", header=False, index=False)